In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import mne
from scipy.signal import butter, lfilter, filtfilt, welch, stft, iirnotch, freqz
from scipy.fft import fft, fftfreq
from sklearn.decomposition import FastICA
import pywt

In [2]:
# Channel Names
n = ['EMG', 'EKG', 'Fp1', 'Fp2', 'A2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4',
     'T4', 'T5', 'P3', 'Pz', 'P4', 'O1', 'O2', 'T1', 'T2', 'PG1', 'PG2', 'A1', 'T6', 'MK']


In [80]:
# Alzheimer's Data
alz_df = []
path = r'/Users/atharvadeshmukh/Desktop/Miniproject_python/alz_csv/'
all_files = glob.glob(os.path.join(path, "*.csv"))
for f in all_files:
    alz_df.append(pd.read_csv(f, names=n))

# Migraine Data
mig_df = []
path = r'/Users/atharvadeshmukh/Desktop/Miniproject_python/mig_csv/'
all_files = glob.glob(os.path.join(path, "*.csv"))
for f in all_files:
    mig_df.append(pd.read_csv(f, names=n))

# Seizure Data
sei_df = []
path = r'/Users/atharvadeshmukh/Desktop/Miniproject_python/seiz_csv/'
all_files = glob.glob(os.path.join(path, "*.csv"))
for f in all_files:
    sei_df.append(pd.read_csv(f, names=n))

# Normal Data
nor_df = []
path = r'/Users/atharvadeshmukh/Desktop/Miniproject_python/nor_csv/'
all_files = glob.glob(os.path.join(path, "*.csv"))
for f in all_files:
    nor_df.append(pd.read_csv(f, names=n))

del all_files
del f
del path

In [82]:

#Initial Clipping 25% Start 25% ending

len1 = len(alz_df)
alz_clip = []
for i in range(len1):
    length = len(alz_df[i])
    mid = length / 2
    cl_point = mid / 2
    upper_lim = int(mid + cl_point)
    lower_lim = int(mid - cl_point)
    clipped = alz_df[i][lower_lim:upper_lim]
    alz_clip.append(clipped)

len1 = len(mig_df)
mig_clip = []
for i in range(len1):
    length = len(mig_df[i])
    mid = length / 2
    cl_point = mid / 2
    upper_lim = int(mid + cl_point)
    lower_lim = int(mid - cl_point)
    clipped = mig_df[i][lower_lim:upper_lim]
    mig_clip.append(clipped)

len1 = len(sei_df)
sei_clip = []
for i in range(len1):
    length = len(sei_df[i])
    mid = length / 2
    cl_point = mid / 2
    upper_lim = int(mid + cl_point)
    lower_lim = int(mid - cl_point)
    clipped = sei_df[i][lower_lim:upper_lim]
    sei_clip.append(clipped)

len1 = len(nor_df)
nor_clip = []
for i in range(len1):
    length = len(nor_df[i])
    mid = length / 2
    cl_point = mid / 2
    upper_lim = int(mid + cl_point)
    lower_lim = int(mid - cl_point)
    clipped = nor_df[i][lower_lim:upper_lim]
    nor_clip.append(clipped)
    
del clipped
del len1
del length
del lower_lim
del upper_lim
del cl_point
del mid
del i
del alz_df
del mig_df
del nor_df
del sei_df


In [83]:

# Initial Lowpass Butterworth filter (6th Order)

def butter_lowpass(lowcut, fs, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    b, a = butter(order, low, btype='low')
    return b, a


def butter_lowpass_filter(data, lowcut, fs, order=6):
    b, a = butter_lowpass(lowcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y


alz_lp = []
for i in range(15):
    x = butter_lowpass_filter(alz_clip[i], 0.5, 128)
    alz_lp.append(x)

mig_lp = []
for i in range(13):
    x = butter_lowpass_filter(mig_clip[i], 0.5, 128)
    mig_lp.append(x)

sei_lp = []
for i in range(10):
    x = butter_lowpass_filter(sei_clip[i], 0.5, 128)
    sei_lp.append(x)

nor_lp = []
for i in range(3):
    x = butter_lowpass_filter(nor_clip[i], 0.5, 128)
    nor_lp.append(x)

del alz_clip
del sei_clip
del mig_clip
del nor_clip

In [53]:
def butter_bandpass(lowcut, highcut, fs=128, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs=128, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Beta and Theta Band Extraction


def bandpass_filter(data):
    out = butter_bandpass_filter(data, 0.5, 60)
    return out

In [77]:
### NOTCH FILTER
    
def notch_filter(data):
    b,a = iirnotch(50, 30, fs=128)
    y = filtfilt(b, a, data)
    return y



In [ ]:
#ICA Decomposition to remove Eye-Blink Artifacts

'''
#This is a sample script, create a seperate script for implementing on each dataset
#Refer https://stackoverflow.com/questions/53071709/remove-eye-blinks-from-eeg-signal-with-ica for details.

ica = FastICA(n_components=27)
ica.fit(#EEGDATA)
components = ica.transform(#EEGDATA)

#Plotting the data

plt.plot(components + [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5])

plt.yticks([0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0, 10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5], ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26'])

plt.ylabel('components')

#To remove the component, example component 2
components[:, 2] = 0

#To reconstruct
restored = ica.inverse_transform(components)

'''

In [ ]:
###########################################################################
## FEATURE EXTRACTION

# Short Time Fourier Transform
def stft_fn(data):
    stft_data = stft(data)
    return stft_data

#To compute for all the data
#Replace all the data_lp by data_ica 

alz_stft = []
for i in range(15):
    x = stft_fn(alz_lp[i])
    alz_stft.append(x)

mig_stft = []
for i in range(13):
    x = stft_fn(mig_lp[i])
    mig_stft.append(x)

sei_stft = []
for i in range(10):
    x = stft_fn(sei_lp[i])
    sei_stft.append(x)

nor_stft = []
for i in range(3):
    x = stft_fn(nor_lp[i])
    nor_stft.append(x)

del alz_lp
del mig_lp
del sei_lp
del nor_lp

In [ ]:
## Power Spectral Density

#Creating a function for PSD

def psd_fn(data):
    f, psd = welch(data,128)
    return psd

#To compute for all the data
#Replace all the data_lp by data_ica 

alz_fft = []
for i in range(15):
    x = psd_fn(alz_lp[i])
    alz_fft.append(x)

mig_fft = []
for i in range(13):
    x = psd_fn(mig_lp[i])
    mig_fft.append(x)

sei_fft = []
for i in range(10):
    x = psd_fn(sei_lp[i])
    sei_fft.append(x)

nor_fft = []
for i in range(3):
    x = psd_fn(nor_lp[i])
    nor_fft.append(x)

del alz_lp
del mig_lp
del sei_lp
del nor_lp


In [84]:
## Discrete Wavelet Transform
## The logic for the function, data is either each channel or each dataset

'''
coeffs = pywt.wavedec(data, 'db4', level=5)
cA5, cD5, cD4, cD3, cD2, cD1 = coeffs
coeffs = cD3, cD5 #Just beta and theta bands
pywt.waverec(coeffs, db4)

'''

def wavelet_fn(data):
    coeffs = pywt.wavedec(data, 'db4', level=5)
    cA5, cD5, cD4, cD3, cD2, cD1 = coeffs
    coeffs = cD3, cD5 #Just beta and theta bands
    wt = pywt.waverec(coeffs, 'db4')
    return wt

#To compute for all the data
#Replace all the data_lp by data_ica 

alz_wt = []
for i in range(15):
    x = wavelet_fn(alz_lp[i])
    alz_wt.append(x)

mig_wt = []
for i in range(13):
    x = wavelet_fn(mig_lp[i])
    mig_wt.append(x)

sei_wt = []
for i in range(10):
    x = wavelet_fn(sei_lp[i])
    sei_wt.append(x)

nor_wt = []
for i in range(3):
    x = wavelet_fn(nor_lp[i])
    nor_wt.append(x)

del alz_lp
del mig_lp
del sei_lp
del nor_lp


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 5 is too high: all coefficients will experience boundary effects.
  warnings.warn(


ValueError: coefficient shape mismatch

In [6]:
#Bandpass without low and high pass
'''
def butter_bandpass(lowcut, highcut, fs=128, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs=128, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Beta and Theta Band Extraction


def betatheta(data):
    a = butter_bandpass_filter(data, 3.5, 7.5)  # Theta Band
    b = butter_bandpass_filter(data, 11.5, 30.5)  # Beta Band
    out = a + b
    return out

'''

In [14]:
# Creating Bandpass filter using lowpass and highpass butterworth filters

def butter_lowpass1(lowcut, fs=128, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    b, a = butter(order, low, btype='low')
    return b, a


def butter_lowpass_filter1(data, lowcut, fs=128, order=4):
    b, a = butter_lowpass1(lowcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def butter_highpass1(highcut, fs=128, order=4):
    nyq = 0.5 * fs
    high = highcut / nyq
    b, a = butter(order, high, btype='high')
    return b, a


def butter_highpass_filter1(data, highcut, fs=128, order=4):
    b, a = butter_highpass1(highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def bandpass_filter(data):
    x1 = butter_lowpass_filter1(data, 3.5)
    x2 = butter_highpass_filter1(data, 7.5)
    theta_band = x1 + x2
    x3 = butter_lowpass_filter1(data, 11.5)
    x4 = butter_highpass_filter1(data, 30.5)
    beta_band = x3 + x4
    out = theta_band + beta_band
    return out




In [15]:
#Bandpass filtering for all the data

alz_bp = []
for i in range(15):
    x = butter_lowpass_filter(alz_lp[i], 0.5, 128)
    alz_bp.append(x)

mig_bp = []
for i in range(13):
    x = butter_lowpass_filter(mig_lp[i], 0.5, 128)
    mig_bp.append(x)

sei_bp = []
for i in range(10):
    x = butter_lowpass_filter(sei_lp[i], 0.5, 128)
    sei_bp.append(x)

nor_bp = []
for i in range(3):
    x = butter_lowpass_filter(nor_lp[i], 0.5, 128)
    nor_bp.append(x)

del alz_lp
del mig_lp
del sei_lp
del nor_lp

In [ ]:
### USING SVM AND KNN TO CHECK INDIVIDUAL CHANNEL ACCURACY 
'''
(MAKE THIS INTO A SINGLE VARIABLE FUNCTION)
'''

### THE DATA IS SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

### SVM MODEL AND ACCURACY
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

svm_mdl = SVC(kernel='linear')
svm_mdl.fit(X_train, y_train)
y_pred = svm_mdl.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

### KNN MODEL AND ACCURACY
from sklearn.neighbors import KNeighborsClassifier

knn_mdl = KNeighborsClassifier(n_neighbors=4)
knn_mdl.fit(X_train, y_train)
y_pred = knn_mdl.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))